# 📘 AM2 Model: Simulation & Calibration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/benmola/OpenAD-lib/blob/main/notebooks/02_AM2_Modelling.ipynb)

This notebook demonstrates the **AM2 (Two-Step Anaerobic Model)** - a simplified 4-state model for anaerobic digestion with **Optuna-based parameter calibration**.

---

## 📚 References

- **AM2 Model & Calibration**: [Dekhici et al. (2024) - ACM DL](https://dl.acm.org/doi/10.1145/3680281)
- **Data-Driven Control**: [Dekhici et al. (2024) - ResearchGate](https://www.researchgate.net/publication/378298857_Data-Driven_Modeling_Order_Reduction_and_Control_of_Anaerobic_Digestion_Processes)
- **Optuna**: [Akiba et al. (2019)](https://arxiv.org/abs/1907.10902), [GitHub](https://github.com/optuna/optuna)

## 🔬 AM2 Model Background

The AM2 model is a simplified representation of anaerobic digestion with **4 state variables**:

| State | Description | Unit |
|-------|-------------|------|
| $S_1$ | Organic substrate (COD) | g COD/L |
| $S_2$ | Volatile Fatty Acids (VFA) | g COD/L |
| $X_1$ | Acidogenic biomass | g/L |
| $X_2$ | Methanogenic biomass | g/L |

### Model Equations

**Mass Balances:**

$$\frac{dS_1}{dt} = D(S_{1,in} - S_1) - k_1 \mu_1(S_1) X_1$$

$$\frac{dS_2}{dt} = D(S_{2,in} - S_2) + k_2 \mu_1(S_1) X_1 - k_3 \mu_2(S_2) X_2$$

$$\frac{dX_1}{dt} = (\mu_1(S_1) - D) X_1$$

$$\frac{dX_2}{dt} = (\mu_2(S_2) - D) X_2$$

### Kinetic Expressions

**Monod Kinetics (Acidogenesis):**

$$\mu_1(S_1) = \frac{\mu_{1,max} \cdot S_1}{K_1 + S_1}$$

**Haldane Kinetics with Substrate Inhibition (Methanogenesis):**

$$\mu_2(S_2) = \frac{\mu_{2,max} \cdot S_2}{K_2 + S_2 + S_2^2/K_i}$$

**Biogas Production:**

$$Q = k_6 \mu_2(S_2) X_2$$

## 1️⃣ Setup (Google Colab)

In [ ]:
# Install OpenAD-lib (uncomment for Colab)
# !pip install "git+https://github.com/benmola/OpenAD-lib.git#egg=openad_lib[optimization]"\n
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:
    sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print(f"Running in Colab: {IN_COLAB}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openad_lib.models.mechanistic import AM2Model, AM2Parameters
from openad_lib.optimisation import AM2Calibrator

print("✅ All imports successful!")

## 2️⃣ AM2 Simulation

First, let's run a simulation with default parameters.

In [ ]:
# Load sample data
if IN_COLAB:
    !wget -q https://raw.githubusercontent.com/benmola/OpenAD-lib/main/src/openad_lib/data/sample_AM2_Lab_data.csv
    data_path = 'sample_AM2_Lab_data.csv'
else:
    base_path = os.path.dirname(os.getcwd())
    data_path = os.path.join(base_path, 'src', 'openad_lib', 'data', 'sample_AM2_Lab_data.csv')

# Initialize model
model = AM2Model()
model.load_data(data_path)

# Display default parameters
print("📊 Default AM2 Parameters:")
print(f"   µ₁ₘₐₓ (m1): {model.params.m1} d⁻¹")
print(f"   K₁:         {model.params.K1} g COD/L")
print(f"   µ₂ₘₐₓ (m2): {model.params.m2} d⁻¹")
print(f"   K₂:         {model.params.K2} g COD/L")
print(f"   Kᵢ:         {model.params.Ki} g COD/L")

In [ ]:
# Run simulation
print("🚀 Running AM2 simulation...")
results = model.run(verbose=True)

# Evaluate
print("\n📈 Evaluation Metrics:")
model.print_metrics()

In [ ]:
# Plot results
model.plot_results(figsize=(12, 10), show_measured=True)

## 3️⃣ Parameter Calibration with Optuna

The `AM2Calibrator` uses **Bayesian optimization** via Optuna to find optimal kinetic parameters.

### Objective Function

$$\text{Minimize: } J = \sum_i w_i \cdot \frac{\text{MSE}_i}{\text{Var}(y_i)}$$

Where:
- $w_i$ = weight for output $i$ (S1, S2, Q)
- $\text{MSE}_i$ = mean squared error
- Normalization by variance ensures balanced contributions

In [ ]:
# Store initial results for comparison
initial_metrics = model.evaluate()
initial_results = results.copy()

# Configure calibrator
calibrator = AM2Calibrator(model)

# Define custom parameter bounds
param_bounds = {
    'm1': (0.01, 0.5),     # Acidogenic growth rate
    'K1': (5.0, 50.0),     # Half-saturation for S1
    'm2': (0.1, 1.0),      # Methanogenic growth rate
    'Ki': (5.0, 50.0),     # Inhibition constant
    'K2': (10.0, 80.0)     # Half-saturation for S2
}

# Define optimization weights
weights = {'S1': 0.5, 'S2': 1.0, 'Q': 1.0}

print("🔧 Calibration Configuration:")
print(f"   Parameters: {list(param_bounds.keys())}")
print(f"   Weights: {weights}")

In [ ]:
# Run calibration (50 trials)
print("🚀 Starting Optuna optimization...")
best_params = calibrator.calibrate(
    params_to_tune=['m1', 'K1', 'm2', 'Ki', 'K2'],
    param_bounds=param_bounds,
    n_trials=50,
    weights=weights,
    show_progress_bar=True
)

In [ ]:
# Run with calibrated parameters
calibrated_results = model.run(verbose=False)
final_metrics = model.evaluate()

# Compare improvements
print("\n📊 Calibration Improvement:")
print("-" * 50)
for var in ['S1', 'S2', 'Q']:
    initial_rmse = initial_metrics[var]['RMSE']
    final_rmse = final_metrics[var]['RMSE']
    pct = (initial_rmse - final_rmse) / initial_rmse * 100
    print(f"{var}: RMSE {initial_rmse:.4f} → {final_rmse:.4f} ({pct:+.1f}%)")

In [ ]:
# Plot comparison
plt.style.use('bmh')
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

variables = ['S1', 'S2', 'Q']
labels = ['COD (S1)', 'VFA (S2)', 'Biogas (Q)']

for i, var in enumerate(variables):
    ax = axes[i]
    
    # Measured data
    if f'{var}_measured' in calibrated_results.columns:
        valid = ~calibrated_results[f'{var}_measured'].isna()
        ax.plot(calibrated_results['time'][valid], 
                calibrated_results[f'{var}_measured'][valid], 
                'o', color='#2E86C1', markersize=6, label='Measured', alpha=0.7)
    
    # Initial model
    ax.plot(initial_results['time'], initial_results[var], 
            '--', color='gray', linewidth=2, label='Initial Model', alpha=0.7)
    
    # Calibrated model
    ax.plot(calibrated_results['time'], calibrated_results[var], 
            '-', color='#27AE60', linewidth=2, label='Calibrated Model')
    
    ax.set_ylabel(labels[i], fontsize=14, fontweight='bold')
    ax.set_title(f'{labels[i]} - Before vs After Calibration', fontsize=16, pad=10)
    ax.legend(fontsize=11)
    ax.grid(True, linestyle='--', alpha=0.7)

axes[-1].set_xlabel('Time (days)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 📝 Summary

This notebook demonstrated:

1. **AM2 Model Structure** - 4-state simplified AD model with Monod and Haldane kinetics
2. **Simulation** - Running the ODE system with default parameters
3. **Calibration** - Bayesian optimization using Optuna for parameter estimation
4. **Validation** - Comparing model predictions before and after calibration

### Next Steps

- Explore [MPC Control](05_MPC_Control.ipynb) for optimal biogas production
- Try [LSTM Surrogate](03_LSTM_Prediction.ipynb) for faster predictions
- Use [Multi-Task GP](04_MTGP_Prediction.ipynb) for uncertainty estimation